In [12]:
import torch
import numpy as np
from models.training import easyTrainer, weights_to_dataset
from models.nODE import nODE, make_nODE_from_parameters
import matplotlib.pyplot as plt
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.loader import DataLoader

In [2]:
adj_list = []
adj_list.append(np.array([[0.,0.],[0.,0.]]))
adj_list.append(np.array([[0,1.],[0,0]]))
adj_list.append(np.array([[0,-1.],[0,0]]))
adj_list.append(np.array([[0,1.],[1.,0]]))
adj_list.append(np.array([[0,1.],[-1.,0]]))
adj_list.append(np.array([[0,-1.],[-1.,0]]))

In [135]:
def generate_data(adj,y,ndata=4):
    ODE_dim = 2
    Gamma = np.array([-1., -1.])
    integration_time = 1
    Win = adj
    Wout = np.array([[2., 0], [0, 2.]])
    bin = np.array([[2.], [2.]])
    bout = np.array([[2.], [2.]])

    train_data, test_data = weights_to_dataset(integration_time, Gamma, Win=Win, bin=bin, Wout=Wout, bout=bout, batch_size = ndata)
    print(train_data)
    edge_index = torch.tensor([[0, 1],
                           [1, 0]], dtype=torch.long)
    
    for i, (x1, x_batch) in enumerate(train_data):
        print(x_batch)
        print(x1)
        x = torch.tensor([[x1[0][0]],[x_batch[0][0]],[x1[0][1]],[x_batch[0][1]],[x1[1][0]],[x_batch[1][0]],[x1[1][1]],[x_batch[1][1]]], dtype=torch.float)

        y = torch.tensor([y])

        data = Data(x= x, edge_index=edge_index,y=y)

    return data

data_list = []
for graph_ind in [0,1,2,3,4,5]:
    for i in range(0,100):
        ind = 0
        data = generate_data(adj_list[graph_ind],graph_ind)
        data_list.append(data)
loader = DataLoader(data_list, batch_size=32, shuffle=True)

test_data_list = []
for graph_ind in [0,1,2,3,4,5]:
    for i in range(0,20):
        ind = 0
        data = generate_data(adj_list[graph_ind],graph_ind)
        test_data_list.append(data)
test_loader = DataLoader(data_list, batch_size=32, shuffle=True)

for data in loader:
    print(data)

tensor([[3.6489, 2.6345],
        [4.0457, 2.5668]])
tensor([[3.1692, 0.4119],
        [4.2478, 0.2277]])
tensor([[3.1214, 3.8596],
        [4.1472, 3.9641]])
tensor([[1.7353, 3.7419],
        [4.5239, 4.0260]])
tensor([[4.1075, 3.3877],
        [2.8591, 4.1372]])
tensor([[4.4159, 2.4592],
        [1.0224, 4.4966]])
tensor([[3.5281, 3.2534],
        [3.5642, 4.1838]])
tensor([[2.8408, 2.0940],
        [2.9389, 4.6232]])
tensor([[2.7344, 2.8098],
        [2.5634, 4.2354]])
tensor([[0.6833, 0.8883],
        [0.2184, 4.7635]])
tensor([[3.9114, 3.1648],
        [3.7626, 3.3399]])
tensor([[3.8829, 1.8533],
        [3.4782, 2.3293]])
tensor([[3.9312, 3.2826],
        [2.5038, 3.9127]])
tensor([[3.9366, 2.1734],
        [0.0566, 3.8863]])
tensor([[2.6295, 2.5165],
        [3.6546, 3.9871]])
tensor([[0.3982, 0.0910],
        [3.1846, 4.0885]])
tensor([[4.1723, 3.2396],
        [2.7880, 3.7818]])
tensor([[4.5921, 2.0566],
        [0.8291, 3.5304]])
tensor([[3.0800, 3.7191],
        [4.0397, 3.6

In [50]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(-1, 16)
        self.conv2 = GATConv(16, 2)
        self.linear = torch.nn.Linear(2, 6)  # Linear layer to output 6x1 vector


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        # Here we aggregate the output from all nodes
        x = x.mean(dim=0, keepdim=True)
        x = self.linear(x)

        return F.log_softmax(x, dim=1)

In [137]:
class fullnet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(8, 10)  # Linear layer to output 6x1 vector
        self.linear2 = torch.nn.Linear(10, 10)  # Linear layer to output 6x1 vector
        self.linear3 = torch.nn.Linear(10, 10)  # Linear layer to output 6x1 vector
        self.linear4 = torch.nn.Linear(10, 6)  # Linear layer to output 6x1 vector

    def forward(self, data):
        x = data.x  # x should be of shape [batch_size, num_features]
        
        # Forward pass through the layers
        x = self.linear(x.t())
        x = torch.relu(x)  # Apply activation
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.linear3(x)
        x = torch.relu(x)
        x = self.linear4(x)  # No activation on the final layer

        return F.log_softmax(x, dim=1)

In [122]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data_list[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)

model.train()
for epoch in range(200):
    loss = 0
    for data_point in data_list:
        data = data_point.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss + F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(1138.7319, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1201.0927, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1101.0800, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1086.5620, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1097.5685, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1085.9463, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1077.5461, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1075.3191, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1075.8744, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1077.8876, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1078.7294, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1078.5189, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1078.9410, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1077.9056, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [138]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = fullnet().to(device)
data = data_list[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    loss = 0
    for data_point in data_list:
        data = data_point.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss + F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(1090.9359, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1087.6859, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1085.0284, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1082.7002, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1080.6356, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1078.8340, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1077.1315, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1075.5056, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1073.9006, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1072.2570, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1070.7252, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1068.9974, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1066.6732, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1063.9330, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1061.0620, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1057.8567, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1053.8810, device='cuda:0', grad_fn=<AddBackward0

In [139]:
correct = 0
total = 0
for data_point in test_data_list:
    data = data_point.to(device)
    out = model(data)
    loss = F.nll_loss(out, data.y)
    pred = torch.argmax(out)
    if pred == data.y[0]:
        correct = correct + 1
    total = total + 1

print(correct/total)

0.6666666666666666
